In [1]:
import pandas as pd
import numpy as np

In [14]:
import seaborn as sns
import matplotlib.pyplot as plt

#Meta Algorithm

In [6]:
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [7]:
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression

In [8]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML 4 course/nsu_train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML 4 course/nsu_test.csv')

Добавим среднюю цену среди категории

In [9]:
avg_price_per_category = []
for cat in np.unique(train_data['Category'].values):
    # print(cat)
    avg_price_per_category.append(train_data[train_data['Category'] == cat]['Average price'].mean())
    cat_mean_price = train_data[train_data['Category'] == cat]['Average price'].mean()
    train_data.loc[train_data['Category'] == cat, 'Avg_price_per_category'] = cat_mean_price
# print(avg_price_per_category)
# train_data.head()

avg_price_per_category = []
for cat in np.unique(test_data['Category'].values):
    # print(cat)
    avg_price_per_category.append(test_data[test_data['Category'] == cat]['Average price'].mean())
    cat_mean_price = test_data[test_data['Category'] == cat]['Average price'].mean()
    test_data.loc[test_data['Category'] == cat, 'Avg_price_per_category'] = cat_mean_price

Добавим фичу, если средняя цена продажи не выше чем средняя цена по сегменту - 1, иначе 0

In [11]:
train_data['avg_price_not_more_than_evg_cat'] = np.where((train_data['Avg_price_per_category'] >= train_data['Average price']), 1, 0)
train_data['avg_price_not_more_than_evg_cat'] = train_data['avg_price_not_more_than_evg_cat'].astype('object')

test_data['avg_price_not_more_than_evg_cat'] = np.where((test_data['Avg_price_per_category'] >= test_data['Average price']), 1, 0)
test_data['avg_price_not_more_than_evg_cat'] = test_data['avg_price_not_more_than_evg_cat'].astype('object')

ratings = train_data['Rating'].values
num_ratings = [int(rating.split(',')[0]) for rating in ratings]
train_data['Rating'] = num_ratings

ratings = test_data['Rating'].values
num_ratings = [int(rating.split(',')[0]) for rating in ratings]
test_data['Rating'] = num_ratings

добавим процентное отклонение BasePrice от AveragePrice

In [12]:
avg_price = train_data['Average price'].values
base_price = train_data['Base price'].values

devitation = np.abs((base_price - avg_price)/avg_price)
print(devitation)
train_data['AvgPrice_BasePrice_percentage_devitation'] = devitation

avg_price = test_data['Average price'].values
base_price = test_data['Base price'].values

devitation = np.abs((base_price - avg_price)/avg_price)
print(devitation)
test_data['AvgPrice_BasePrice_percentage_devitation'] = devitation

[0.08266153 1.44055069 0.05272727 ... 0.07336695 0.96251378 1.56508219]
[1.70381587 1.00814111 0.46786031 ... 0.67283991 0.5625     0.1765061 ]


In [13]:
train_data['comments_more_than_avg'] = (train_data['Comments'] > 4).astype('object')
# pd.crosstab(train_data['comments_more_than_avg'], train_data['Sales'], margins=True)
test_data['comments_more_than_avg'] = (test_data['Comments'] > 4).astype('object')

In [15]:
def deleteId(ds):
    return ds.drop(columns='Id')

def deleteNaN(train_ds, test_ds, critval):
    fullsize = train_ds.shape[0]
    new_train = train_ds.copy()
    new_test = test_ds.copy()
    for feature in new_train.columns:
        nulls = new_train[feature].isnull().sum()
        percent = nulls / fullsize
        # если доля пустых значений превышает critval - столбец не информативен,
        # можно его выбросить
        if (percent > critval):
            new_train = new_train.drop(columns=feature)
            new_test = new_test.drop(columns=feature)
    return new_train, new_test

def convertToNumeric(train_ds, test_ds):
    new_train = train_ds.copy()
    new_test = test_ds.copy()
    LE = LabelEncoder()
    for feature in new_train.columns:
        if (new_train[feature].dtype == 'object'):
            new_train[feature] = LE.fit_transform(new_train[feature])
            new_test[feature] = LE.fit_transform(new_test[feature])
    return new_train, new_test

In [16]:
train = train_data.drop_duplicates()

# train = train.drop(columns='Id')
train = train.drop(columns='Color')
# train = train.drop(columns='Base price')
# train = train.drop(columns='Basic Sale Price')
train = train.drop(columns='Max price')
train = train.drop(columns='Min price')

test = test_data.drop(columns='Id')
test = test.drop(columns='Color')
# test = test.drop(columns='Base price')
# test = test.drop(columns='Basic Sale Price')
test = test.drop(columns='Max price')
test = test.drop(columns='Min price')

train, test = deleteNaN(train, test, critval=0.8)
train, test = convertToNumeric(train, test)

train_y = train['Sales'].values
train_X = train.drop(columns='Sales').values
val_test = test.values

train_x_models, train_x_meta, train_y_models, train_y_meta = train_test_split(train_X, train_y, test_size=0.3, random_state=7)

train_x, test_x, train_y, test_y = train_test_split(train_x_models, train_y_models, test_size=0.1, random_state=7)  # random_state=98987)
train_x_meta, test_x_meta, train_y_meta, test_y_meta = train_test_split(train_x_meta, train_y_meta, test_size=0.1, random_state=7)

In [19]:
strategies = ['mean', 'median', 'most_frequent']
imputer = SimpleImputer(strategy=strategies[2])
trainX = imputer.fit_transform(train_x)
testX = imputer.fit_transform(test_x)

imputer = SimpleImputer(strategy=strategies[2])
trainX_meta = imputer.fit_transform(train_x_meta)
testX_meta = imputer.fit_transform(test_x_meta)

imputer = SimpleImputer(strategy=strategies[2])
val_test_x = imputer.fit_transform(val_test)

In [20]:
scaler = StandardScaler()
trainX = scaler.fit_transform(trainX)
testX = scaler.transform(testX)

scaler = StandardScaler()
trainX_meta = scaler.fit_transform(trainX_meta)
testX_meta = scaler.transform(testX_meta)

scaler = StandardScaler()
val_test_x = scaler.fit_transform(val_test_x)

#Метрика **SMAPE**

In [21]:
def smape(A, F):
    with np.errstate(divide='ignore', invalid='ignore'):
        tmp = 2 * np.abs(F-A) / (np.abs(A) + np.abs(F))
    tmp[np.isnan(tmp)] = 0
    return np.sum(tmp) / len(tmp) * 100

def check_error(preds, gt):
    print('SMAPE Error:', smape(np.round(preds), gt))

#Random Forest

обучим сразу несколько моделей, а потом по их предсказаниям построим регрессор, который приблизит эти значения до нужно уровня

In [24]:
print(trainX.shape)

(15460, 18)


In [22]:
best_parameters = {'criterion': 'squared_error', 
                  'max_depth': 3000, 
                  'min_samples_leaf': 7, 
                  'min_samples_split': 4, 
                  'n_estimators': 70}  # - submission(8) - 20.89%

model1 = RandomForestRegressor(**best_parameters) 
model1.fit(trainX, train_y)
check_error(model1.predict(testX), test_y)

SMAPE Error: 25.72874118204595


In [25]:
best_parameters = {'criterion': 'squared_error',
                  'max_depth': 3000,
                  'min_samples_leaf': 7,
                  'min_samples_split': 4,
                  'n_estimators': 80}  # - submission(8) - 20.87%

# random_forest = RandomForestRegressor(**rf_gs_model.best_params_)
model2 = RandomForestRegressor(**best_parameters) 
model2.fit(trainX, train_y)
check_error(model2.predict(testX), test_y)

SMAPE Error: 25.66257482447715


In [26]:
best_parameters = {'criterion': 'squared_error',
                  'max_depth': 3000, 
                  'min_samples_leaf': 7,
                  'min_samples_split': 4,
                  'n_estimators': 100}  # - submission(9) - 20.92%

# random_forest = RandomForestRegressor(**rf_gs_model.best_params_)
model3 = RandomForestRegressor(**best_parameters) 
model3.fit(trainX, train_y)
check_error(model3.predict(testX), test_y)

SMAPE Error: 25.837325684078206


In [27]:
best_parameters = {'criterion': 'squared_error',
                  'max_depth': 3500,
                  'min_samples_leaf': 7,
                  'min_samples_split': 4,
                  'n_estimators': 100}  # - submission(10) - 20.77%

# random_forest = RandomForestRegressor(**rf_gs_model.best_params_)
model4 = RandomForestRegressor(**best_parameters) 
model4.fit(trainX, train_y)
check_error(model4.predict(testX), test_y)

SMAPE Error: 25.73733541354521


In [28]:
best_parameters = {'criterion': 'squared_error',
                  'max_depth': 4000, 
                  'min_samples_leaf': 7,
                  'min_samples_split': 4,
                  'n_estimators': 80}  # - submission(11) - 20.88%

# random_forest = RandomForestRegressor(**rf_gs_model.best_params_)
model5 = RandomForestRegressor(**best_parameters) 
model5.fit(trainX, train_y)
check_error(model5.predict(testX), test_y)

SMAPE Error: 25.940215408068184


In [29]:
best_parameters = {'criterion': 'squared_error', 
                  'max_depth': 4000, 
                  'min_samples_leaf': 7,
                  'min_samples_split': 4,
                  'n_estimators': 100}  # - submission(12) - 20.85%

# random_forest = RandomForestRegressor(**rf_gs_model.best_params_)
model6 = RandomForestRegressor(**best_parameters) 
model6.fit(trainX, train_y)
check_error(model6.predict(testX), test_y)

SMAPE Error: 26.055028981679033


In [32]:
best_parameters = {
    'criterion': 'absolute_error', 
    'max_depth': 4000,  # 300
    'n_estimators': 120, # 70
    'min_samples_leaf': 7, # 3
    'min_samples_split': 4} # 50

model7 = RandomForestRegressor(**best_parameters) 
model7.fit(trainX, train_y)
check_error(model7.predict(testX), test_y)

SMAPE Error: 19.777093325745685


###Meta

In [36]:
trainX_meta.shape

(6625, 18)

In [40]:
predictions1 = np.round(model1.predict(trainX_meta))
predictions2 = np.round(model2.predict(trainX_meta))
predictions3 = np.round(model3.predict(trainX_meta))
predictions4 = np.round(model4.predict(trainX_meta))
predictions5 = np.round(model5.predict(trainX_meta))
predictions6 = np.round(model6.predict(trainX_meta))
predictions7 = np.round(model7.predict(trainX_meta))
train_samples = np.vstack((predictions1, predictions2, predictions3, predictions4, predictions5, predictions6, predictions7)).T
print(train_samples.shape)

(6625, 7)


In [44]:
# lgr = LogisticRegression(max_iter=1000)
# lgr.fit(train_samples, train_y_meta)
best_parameters = {
    'criterion': 'absolute_error', 
    'max_depth': 4500,  # 300
    'n_estimators': 50, # 70
    'min_samples_leaf': 7, # 3
    'min_samples_split': 4} # 50

meta_model = RandomForestRegressor(**best_parameters) 
meta_model.fit(train_samples, train_y_meta)
# check_error(model7.predict(testX), test_y)
predictions1 = np.round(model1.predict(testX_meta))
predictions2 = np.round(model2.predict(testX_meta))
predictions3 = np.round(model3.predict(testX_meta))
predictions4 = np.round(model4.predict(testX_meta))
predictions5 = np.round(model5.predict(testX_meta))
predictions6 = np.round(model6.predict(testX_meta))
predictions7 = np.round(model7.predict(testX_meta))
test_samples = np.vstack((predictions1, predictions2, predictions3, predictions4, predictions5, predictions6, predictions7)).T
print(test_samples.shape)
check_error(meta_model.predict(test_samples), test_y_meta)

SMAPE Error: 19.777093325745685
(737, 7)
SMAPE Error: 21.906788247908672


In [45]:
predictions1 = np.round(model1.predict(val_test_x))
predictions2 = np.round(model2.predict(val_test_x))
predictions3 = np.round(model3.predict(val_test_x))
predictions4 = np.round(model4.predict(val_test_x))
predictions5 = np.round(model5.predict(val_test_x))
predictions6 = np.round(model6.predict(val_test_x))
predictions7 = np.round(model7.predict(val_test_x))
val_samples = np.vstack((predictions1, predictions2, predictions3, predictions4, predictions5, predictions6, predictions7)).T
predictions = np.round(meta_model.predict(val_samples))

In [47]:
submission_path = '/content/drive/MyDrive/Colab Notebooks/ML 4 course/nsu_sample_solution.csv'
submission = pd.read_csv(submission_path)

submission['Expected'] = predictions
submission
submission.to_csv('submission_meta.csv', index=False)

#Просто среднее предсказание лучших алгоритмов

In [49]:
pred1 = np.round(model1.predict(val_test_x))
pred2 = np.round(model2.predict(val_test_x))
pred3 = np.round(model3.predict(val_test_x))
pred4 = np.round(model4.predict(val_test_x))
pred5 = np.round(model5.predict(val_test_x))
pred6 = np.round(model6.predict(val_test_x))
pred7 = np.round(model7.predict(val_test_x))

amount = 7
print(len(best_pred))
prediction = (pred1 + pred2 + pred3 + pred4 + pred5 + pred6 + pred7) / 7
print(len(prediction))

10741
10741


In [5]:
submission_path = '/content/drive/MyDrive/Colab Notebooks/ML 4 course/nsu_sample_solution.csv'
submission = pd.read_csv(submission_path)

submission['Expected'] = np.round(prediction)
submission
submission.to_csv('best_submission.csv', index=False)